# Setup and Imports

In [1]:
%%capture
!rm -rf fairgraph
!git clone https://github.com/GMattheisen/fairgraph.git
!pip install -r ./fairgraph/requirements.txt
!pip install -U ./fairgraph

In [2]:
from fairgraph import KGClient
import os
import re
import io
import logging
from datetime import date, datetime
from pprint import pprint
import yaml
from collections import defaultdict
import requests
import numpy as np
from neo.io import get_io, Spike2IO, NeoMatlabIO, RawBinarySignalIO, NeoHdf5IO
from fairgraph import KGClient
from fairgraph.commons import (Species, Handedness, Sex, Strain, Age, Genotype, QuantitativeValue, 
    BrainRegion, CellType, QuantitativeValueRange, StimulusType)
from fairgraph.core import (Person, Identifier, Material, Organization, Address, Subject, Protocol, Step, 
    use_namespace)
from fairgraph.experiment import (Device, ElectrophysiologicalStimulation,  
    ElectrophysiologicalStimulus, CranialWindow, Craniotomy)
from fairgraph.electrophysiology import (PatchedCell, PatchedCellCollection, 
    PatchClampActivity, PatchClampExperiment, Trace, MultiChannelMultiTrialRecording,
    QualifiedMultiTraceGeneration)
from fairgraph.base import KGQuery, as_list, Distribution
from fairgraph.minds import Dataset

use_namespace('neuralactivity')

/Users/glynismattheisen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
logging.basicConfig(filename="to_knowledge_graph.log",
                    filemode='a',
                    level=logging.DEBUG)

logger = logging.getLogger("nar")

token = ""
client = KGClient(token, nexus_endpoint='https://nexus-int.humanbrainproject.org/v0')

# direct paths dependent on environment
if 'int' in client.nexus_endpoint:
    Subject._path = '/core/subject/v0.1.2'
    Slice._path = '/core/slice/v0.1.0'
    PatchedCell._path = '/experiment/patchedcell/v0.1.0'
    Material._path = '/core/material/v0.1.2'
    Step._path = '/core/protocol/v0.1.6'
    Protocol._path = '/core/protocol/v0.1.6'
    ElectrophysiologicalStimulus._path = '/optophysiology/electrophysiologicalstimulus/v0.1.0'
    ElectrophysiologicalStimulation._path = '/optophysiology/electrophysiologicalstimulation/v0.5.0'
    Device._path = '/electrophysiology/device/v0.1.2'
    PatchClampExperiment._path = '/electrophysiology/stimulusexperiment/v0.3.1'
    QualifiedMultiTraceGeneration._path = '/electrophysiology/multitracegeneration/v0.2.3'
    MultiChannelMultiTrialRecording._path = '/electrophysiology/multitrace/v0.3.1'
    PatchedCellCollection._path = '/experiment/patchedcellcollection/v0.1.0'
    PatchedSlice._path = '/experiment/patchedslice/v0.1.0'
    PatchClampActivity._path = '/experiment/wholecellpatchclamp/v0.4.0'
    BrainSlicingActivity._path = '/experiment/brainslicing/v0.1.0'

### Whole Cell Patch Clamp In Vivo

#### Components

Subject<br>
PatchedCell<br>
CranialWindow<br>
Craniotomy<br>
Step<br>
Protocol<br>
ElectrophysiologicalStimulus<br>
ElectrophysiologicalStimulation<br>
Device<br>
PatchClampExperiment<br>
QualifiedMultiTraceGeneration<br>
MultiChannelMultiTrialRecording<br>
PatchClampActivity

# Locate Dataset and Define Contributors

In [4]:
dataset_name = None

minds_dataset = Dataset.by_name(dataset_name, client, resolved=True, api="query")

# map MINDSPerson to Person (for owner, contributors)
owner = [obj.resolve(client) for obj in as_list(minds_dataset.owners)]
contributors = [obj.resolve(client) for obj in as_list(minds_dataset.contributors)]

In [ ]:
for minds_person in contributors:
    print(minds_person)

In [5]:
"""People"""
abc = Person(given_name="", family_name="", email="")
abc.save(client)
deg = Person(given_name="", family_name="", email=None)
deg.save(client)
hij = Person(given_name="", family_name="", email=None)
hij.save(client)

people = [abc, deg, hij]

specimen_group = [obj.resolve(client) for obj in as_list(minds_dataset.specimen_group)]
minds_subjects = []
for sg in specimen_group:
    minds_subjects.extend([obj.resolve(client) for obj in as_list(sg.subjects)])

# Define Subjects, Samples, Experiment

In [7]:
for minds_subject in minds_subjects:
    print(minds_subject.name)
    print(minds_subject.samples)
    for minds_sample in as_list(minds_subject.samples):
        print(as_list(minds_subject.samples))
        minds_sample = minds_sample.resolve(client)
        print(minds_sample.name)

"with open(f'{minds_dataset.name}_trace_dict.yaml') as file:\n    trace_dict = yaml.load(file, Loader=yaml.FullLoader)\n    print(trace_dict)"

In [8]:
"""Import trace information from previous run if available"""

if os.path.exists(f'{minds_dataset.name}_trace_dict.yaml'):
    print("loading trace_dict")
    with open(f'{minds_dataset.name}_trace_dict.yaml') as file:
        trace_dict = yaml.load(file, Loader=yaml.FullLoader)
        preload = True
        print(trace_dict)
else:
    print("no saved trace_dict, building trace_dict")
    trace_dict = {}
    preload = False

hbp-00806.HBP_01
hbp-00806.HBP_02
hbp-00806.HBP_03
hbp-00806.HBP_04
hbp-00806.HBP_05
hbp-00806.HBP_06
hbp-00806.HBP_07
hbp-00806.HBP_08
hbp-00806.HBP_09
hbp-00806.HBP_10
hbp-00806.HBP_11
hbp-00806.HBP_12
hbp-00806.HBP_13
hbp-00806.HBP_14
hbp-00806.HBP_15
hbp-00806.HBP_16
hbp-00806.HBP_17
hbp-00806.HBP_18
hbp-00806.HBP_19
hbp-00806.HBP_20
hbp-00806.HBP_21
hbp-00806.HBP_22
hbp-00806.HBP_23
hbp-00806.HBP_24
hbp-00806.HBP_25
hbp-00806.HBP_26
hbp-00806.HBP_27
hbp-00806.HBP_28


In [1]:
# map MINDSSubject to Subject
for minds_subject in minds_subjects:
    print(minds_subject.name)

    """Subject"""
    species = minds_subject.species.resolve(client).name
    strain = minds_subject.strains
    genotype = minds_subject.genotype
    sex = minds_subject.sex.resolve(client).name
    if sex == "Unknown" or sex == '<Unknown>':
        sex = None
    else:
        sex=Sex(sex)
    handedness = None
    age_units = "days"
    age_value = minds_subject.age
    if "-" in age_value:
        min, max = age_value.split("-")
        age_value = QuantitativeValueRange(int(min), int(max), age_units)
    else:
        age_value = QuantitativeValue(int(age_value), age_units)    
    age_category = "Post-natal"
    death_date = None # date type
    group = None # 'control group' v 'treatment group'

    subject = Subject(
      minds_subject.name,
      species=Species(species),
      strain=Strain(strain),
      genotype=Genotype(genotype),
      sex = sex,
      age=Age(age_value, age_category),
      death_date=None
      )
    print(subject)
    subject.save(client)
    assert subject.id is not None
            
    slices = []
    cells = []
    for minds_sample in as_list(minds_subject.samples):
        minds_sample = minds_sample.resolve(client)
        
        brain_regions = [BrainRegion("hippocampus CA1")]  
        
        patched_cell = PatchedCell(
            name = f"Patched cell in slice for {minds_sample.name}",
            brain_location = brain_regions,
            cell_type=CellType(""),
            morphology_type = MorphologyType(""),
            pipette_id = "", # string or int
            seal_resistance = QuantitativeValue(int(value), "units"),
            pipette_resistance = QuantitativeValue(int(value), "units"),
            liquid_junction_potential = QuantitativeValue(int(value), "units"),
            start_membrane_potential = QuantitativeValue(int(value), "units"),
            end_membrane_potential = QuantitativeValue(int(value), "units"),
            pipette_solution = "", 
            labeling_compound = "", 
            reversal_potential_cl = QuantitativeValue(int(value), "units"),
            description = ""
            )
        patched_cell.save(client)        
        cells.append(patched_cell)
        print(cells)

        cranial_window = CranialWindow(
            name = f"Cranial window for {subject.name}",
            brain_location = brain_location,
            window_type = "",
            diameter = QuantitativeValue(int(value), "units"),
            fluorescence_labeling = "",
            description = ""
        )
        cranial_window.save(client)
    
        craniotomy = Craniotomy(
            name = f"Craniotomy for {subject.name}",
            subject = subject,
            cranial_window = cranial_window,
            anesthesia = "",      
            protocol = Protocol(),
            start_time =, # datetime
            end_time = , # datetime
            people = people
        )
        craniotomy.save(client)

        step_one = Step(
            name = "Step one for protocol",
            previous_step_name = None,
            sequence_number = 1,
            version = "", # string or int
            identifier = "",
            distribution = Distribution(""),
            description = "",
            materials = Material(), # or list
            author = people 
        )
        step_one.save(client)
            
        step_two = Step(
            name = "Step two for protocol",
            previous_step_name = "Step one for protocl",
            sequence_number = 2,
            version = "", # string or int
            identifier = "",
            distribution = Distribution(""),
            description = "",
            materials - Material(), # or list
            author = people
        )
        step_two.save(client)

        steps = [step_one, step_two]
        
        protocol = Protocol(
            name = "Protcol for stimulation",
            version = "", # string or int
            identifier = "",
            distribution = Distribution(),
            number_of_steps = 1,
            steps = steps, 
            materials = Material(), # or list
            author = Person(), # or list
            date_published = # datetime
        )
        protocol.save(client)
    
        electrophysiological_stimulus = ElectrophysiologicalStimulus(
            name=f"Stimulus for patched cell in {minds_sample.name}",
            description = "",
            distribution = None
            )
        electrophysiological_stimulus.save(client)

        electrophysiological_stimulation = ElectrophysiologicalStimulation(
            name=f"Stimulation for patched cell in {minds_sample.name}",
            stimulus = electrophysiological_stimulus,
            protocol= protocol,
            citation = None,
            code = None,
            license = None
            )
        electrophysiological_stimulation.save(client)
        print(electrophysiological_stimulation)

        device = Device(
            name = "",
            manufacturer="",
            model_name="",
            software_version =None,
            serial_number = None,
            distribution = Distribution(""),
            )
        device.save(client)
        print(device)
        
        expt = PatchClampExperiment(
            name=f"Whole cell recordings in patched cell for {minds_sample.name}",
            recorded_cell= patched_cell,
            acquisition_device = device,
            stimulation = electrophysiological_stimulation,
            start_time = None,
            end_time = None,
            people = people,
            protocol=None
            )
        expt.save(client)
        print(expt)
        
        """Trace Generation"""
        """Accounts for muliple files existing per sample"""
        
        holding_potential_value = -70
        holding_potential_unit = 'mV'
        
        if preload: # building from YAML
            print("building trace metadata from YAML")
            trace_generation = QualifiedMultiTraceGeneration(
                f"Metadata for whole cell recordings in {minds_sample.name}",
                stimulus_experiment=expt,
                sweeps=trace_dict[minds_sample.name]['n_segments'],
                channel_type = None,
                holding_potential = QuantitativeValue(holding_potential_value, holding_potential_unit),
                sampling_frequency = None, 
                power_line_frequency = None
                )
            trace_generation.save(client)
            print(trace_generation)

            trace = MultiChannelMultiTrialRecording(
                f"Traces recorded in {minds_sample.name}",
                data_location=Distribution(location = trace_dict[minds_sample.name]['file_location'],
                  content_type=trace_dict[minds_sample.name]['content_type']),
                generated_by=expt,
                generation_metadata=trace_generation,
                channel_names=trace_dict[minds_sample.name]['channel_names'],
                data_unit=trace_dict[minds_sample.name]['data_unit'],
                time_step=QuantitativeValue(trace_dict[minds_sample.name]['time_step'], trace_dict[minds_sample.name]['time_step_unit']),
                channel_type = None,
                part_of=minds_dataset
                )
            print(trace)
            trace.save(client)
        
        else:
            file_dir = "hbp-d00000/"
            filename = os.listdir(file_dir)
            file_url=file_dir + filename
            file_location = "https://object.cscs.ch/v1/AUTH_0000000000000000000/" + file_url

            io = RawBinarySignalIO(filename = file_url)
            data = io.read_block()
            n_segments = len(data.segments)
            sigs = data.segments[0].analogsignals
            channel_names = [sig.name for sig in sigs]
            units = [sig.units.dimensionality.string for sig in sigs]
            time_step_unit = "ms"
            time_step = float(sigs[0].sampling_period.rescale(time_step_unit))
            content_type = ""
            
            print("trace_dict does not exist, building trace dict") #
            trace_dict[minds_sample.name] = {}
            trace_dict[minds_sample.name]['channel_names'] = channel_names
            trace_dict[minds_sample.name]['data_unit'] = units
            trace_dict[minds_sample.name]['time_step'] = time_step
            trace_dict[minds_sample.name]['time_step_unit'] = time_step_unit
            trace_dict[minds_sample.name]['n_segments'] = n_segments
            trace_dict[minds_sample.name]['file_url'] = file_url
            trace_dict[minds_sample.name]['file_location'] = file_location
            trace_dict[minds_sample.name]['content_type'] = content_type

            trace_generation = QualifiedMultiTraceGeneration(
                f"Metadata for whole cell recordings in {minds_sample.name}",
                stimulus_experiment = expt,
                sweeps = n_segments,
                channel_type = None,
                holding_potential = QuantitativeValue(holding_potential_value, holding_potential_unit),
                sampling_frequency = None, 
                power_line_frequency = None
                )
            trace_generation.save(client)
            print(trace_generation)

            trace = MultiChannelMultiTrialRecording(
                f"Traces recorded in patched cell for {minds_sample.name}",
                data_location=Distribution(location = file_location,
                  content_type=content_type),
                generated_by=expt,
                generation_metadata=trace_generation,
                channel_names=channel_names,
                data_unit=units,
                time_step=QuantitativeValue(**{
                "unit_text": time_step_unit,
                "value": time_step
                }),
                channel_type = None,
                part_of=minds_dataset
                )
            print(trace)
            trace.save(client)
                
    patch_clamp_activity = PatchClampActivity(
        name = f"Patch clamp activity for patched cell in {minds_subject.name}",
        recorded_tissue = slice,
        recorded_slice = patched_slice,
        protocol = None,
        people = people,
        start_time = None, # datetime format
        end_time = None # datetime format
        )
    patch_clamp_activity.save(client)
    print(patch_clamp_activity)       

SyntaxError: invalid syntax (<ipython-input-1-b2614e752d5a>, line 82)

In [ ]:
trace_dict

In [ ]:
with open(f'{minds_dataset.name}_trace_dict.yaml', 'w') as file: #save trace dict
    documents = yaml.dump(trace_dict, file)

In [10]:
print(f"Run on {datetime.now()} on{client.nexus_endpoint}")

Run on 2020-06-23 10:38:24.995896 onhttps://nexus-int.humanbrainproject.org/v0
